<a href="https://colab.research.google.com/github/Onoruoiza514/Playwright-automation-suite-/blob/main/FMSCA_scraper_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 16.0 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 29.4s161.3 MiB [] 0% 18.9s161.3 MiB [] 0% 11.7s161.3 MiB [] 1% 5.8s161.3 MiB [] 1% 4.2s161.3 MiB [] 2% 3.4s161.3 MiB [] 3% 2.9s161.3 MiB [] 4% 2.5s161.3 MiB [] 5% 2.4s161.3 MiB [] 6% 2.3s161.3 MiB [] 6% 2.5s161.3 MiB [] 7% 2.4s161.3 MiB [] 8% 2.3s161.3 MiB [] 9% 2.2s161.3 MiB [] 10% 2.2s161.3 MiB [] 11% 2.1s161.3 MiB [] 12% 2.0s161.3 MiB [] 14% 1.9s161.3 MiB [] 15% 1.8s161.3 MiB [] 16% 1.8s161.3 MiB [] 18% 1.7s161.3 MiB [] 19% 1.6s161.3 MiB [] 20% 1.6s161.3 MiB [] 21% 1.5s161.3 MiB [] 22% 1.5s161.3 MiB [] 23% 1.5s161.3 MiB [] 24% 1.5s161.3 MiB [] 25% 1.5s161.3 MiB [] 26% 1.4s161.3 MiB [] 27% 1.4s161.3 MiB [] 28% 1.3s161.3 MiB [] 29% 1.4s161.3 MiB [] 30% 1.4s161.3 MiB [] 31% 1.3s161.3 MiB [] 32% 1.3s161.3 MiB [] 34% 1.2s161.3 MiB [] 35% 1.2s161.3 MiB [] 36% 1.2s161.3 MiB [] 37% 1.1s161.3 MiB [] 39% 1.1s161.3 MiB [] 40% 1.1s161.3 MiB [] 41% 1.1s161.3 MiB [] 42% 1.1s161.3 MiB

In [ ]:
#%%writefile links_scraper_for_carrier_companies.py
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_company_list():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Navigate to the FMCSA page
        await page.goto("https://safer.fmcsa.dot.gov/keywordx.asp?searchstring=%2AAN%2A&SEARCHTYPE=")
        await page.screenshot(path="example.png")
        # Wait for the page to load
        await page.wait_for_selector("tr")

        # Extract data
        rows = await page.query_selector_all("tr")
        data = []

        for row in rows:
            try:
                # Extract company name, location, and href
                company_element = await row.query_selector("th > b > a")
                location_element = await row.query_selector("td > b")

                company_name = await company_element.inner_text() if company_element else "N/A"
                location = await location_element.inner_text() if location_element else "N/A"
                link = await company_element.get_attribute("href") if company_element else "N/A"

                # Add to the data list
                data.append({
                    "Company Name": company_name.strip(),
                    "Location": location.strip(),
                    "Link": f"https://safer.fmcsa.dot.gov/{link.strip()}" if link != "N/A" else "N/A"
                })
            except Exception as e:
                print(f"Error processing row: {e}")

        # Save to CSV
        df = pd.DataFrame(data)
        df.to_csv("company_data.csv", index=False)

        await browser.close()
        print("Scraping complete. Data saved to 'company_data.csv'.")

# Run the script
import asyncio
await scrape_company_list()

Scraping complete. Data saved to 'company_data.csv'.


In [ ]:
import pandas as pd
df = pd.read_csv("company_data.csv")
df.head(30)


,Company Name,Location,Link
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,1998 TRANSPORT,"ALTAIR, TX",https://safer.fmcsa.dot.gov/query.asp?searchty...
6,3 M TRANSPORT,"GRETHEL, KY",https://safer.fmcsa.dot.gov/query.asp?searchty...
7,3-G TRANSPORT,"FLORENCE, SC",https://safer.fmcsa.dot.gov/query.asp?searchty...
8,A & I TRANSPORT CORP,"HIALEAH, FL",https://safer.fmcsa.dot.gov/query.asp?searchty...
9,A A EAGLE PUMPING & DRAIN CLEANING INC,"HUDSON, FL",https://safer.fmcsa.dot.gov/query.asp?searchty...


In [ ]:
import asyncio
import csv
from playwright.async_api import async_playwright

async def scrape_company_data(input_csv, output_csv):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # List to store scraped data
        scraped_data = []

        # Read URLs from the input CSV file
        urls = read_urls_from_csv(input_csv) #We would create a helper function that would go onto our earlier scraped file and

        # Iterate through each URL
        for index, url in enumerate(urls, start=1):
            try:
                # Visit the URL
                await page.goto(url, timeout=30000)
                await page.screenshot(path="Test_photo.png")
                print("Navigated to the page..........")

                # Check if the page contains the expected table
                table_exists = page.locator('table[border="1" width="70%" bordercolor="SILVER" cellspacing="0" cellpadding="4" summary="For formatting purpose"]')
                if not table_exists:
                    print(f"Link number {index} is not available.")
                    scraped_data.append({
                        "Entity Type": "N/A",
                        "USDOT Status": "N/A",
                        "USDOT Number":"N/A",
                        "MCS-150 Mileage Year":"N/A",
                        "MCS-150 Form Date":"N/A",
                        "Legal Name":"N/A",
                        "DBA Name":"N/A",
                        "Physical Address":"N/A",
                        "Phone":"N/A",
                        "Mailing Address":"N/A",
                        "URL":url,
                        "Error": f"Link Number {index} not available......."
                    })
                    continue

                # Scrape data from the table
                data = {
                    "Entity Type": await extract_text(page, "th:has-text('Entity Type:') + td"),
                    "USDOT Status": await extract_text(page, "th:has-text('USDOT Status:') + td"),
                    "USDOT Number": await extract_text(page, "th:has-text('USDOT Number:') + td"),
                    "MCS-150 Mileage Year": await extract_text(page, "th:has-text('MCS-150 Mileage (Year):') + td"),
                    "MCS-150 Form Date": await extract_text(page, "th:has-text('MCS-150 Form Date:') + td"),
                    "Legal Name": await extract_text(page, "th:has-text('Legal Name:') + td"),
                    "DBA Name": await extract_text(page, "th:has-text('DBA Name:') + td"),
                    "Physical Address": await extract_text(page, "th:has-text('Physical Address:') + td"),
                    "Phone": await extract_text(page, "th:has-text('Phone:') + td"),
                    "Mailing Address": await extract_text(page, "th:has-text('Mailing Address:') + td"),
                    "URL": url,
                    "Error": "N/A"
                }

                scraped_data.append(data)

            except Exception as e:
                print(f"Error scraping link number {index}: {e}")
                scraped_data.append({
                    "Entity Type": "N/A",
                    "USDOT Status":"N/A",
                    "USDOT Number": "N/A",
                    "MCS-150 Mileage Year": "N/A",
                    "MCS-150 Form Date": "N/A",
                    "Legal Name":"N/A",
                    "DBA Name" : "N/A",
                    "Physical Address": "N/A",
                    "Phone" : "N/A",
                    "Mailing Address": "N/A",
                    "URL": url,
                    "Error": f"Error scraping link number {index}: {e}"
                })

        # Save data to the output CSV file
        save_to_csv(scraped_data, output_csv) #We would also create am additional functions to sace our scraped datas

        await browser.close()

def read_urls_from_csv(filename):
    """Read URLs from a CSV file."""
    urls = []
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            urls.append(row['Link'])
            #Print we got the links successfully
    return urls

def save_to_csv(data, filename):
    """Save data to a CSV file."""
    fieldnames = ["Entity Type","USDOT Number","USDOT Status","MCS-150 Mileage Year","MCS-150 Form Date","Legal Name","DBA Name","Physical Address","Phone","Mailing Address","URL","Error"]

    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

async def extract_text(page, selector):
    """Extract text from a selector."""
    element = await page.query_selector(selector)
    return await element.inner_text() if element else "N/A"

# Example usage
input_csv = "company_data.csv"  # CSV file containing the URLs inq a column named 'URL'
output_csv = "scraped_data.csv"

# Run the scraper
await scrape_company_data(input_csv, output_csv)

Error scraping link number 1: Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
  - navigating to "N/A", waiting until "load"

Error scraping link number 2: Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
  - navigating to "N/A", waiting until "load"

Error scraping link number 3: Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
  - navigating to "N/A", waiting until "load"

Error scraping link number 4: Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
  - navigating to "N/A", waiting until "load"

Error scraping link number 5: Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
  - navigating to "N/A", waiting until "load"

Navigated to the page..........
Navigated to the page..........
Navigated to the page..........
Navigated to the page..........
Navigated to the page..........
Navigated to the page..........
Nav

**Now we have to loop through all these links we've scrapped earlier then we extract necessary information we need from each,so help us God**

In [ ]:
import pandas as pd
oga = pd.read_csv(output_csv)
oga.head(30)

,Entity Type,USDOT Number,USDOT Status,MCS-150 Mileage Year,MCS-150 Form Date,Legal Name,DBA Name,Physical Address,Phone,Mailing Address,URL,Error
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error scraping link number 1: Page.goto: Proto...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error scraping link number 2: Page.goto: Proto...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error scraping link number 3: Page.goto: Proto...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error scraping link number 4: Page.goto: Proto...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error scraping link number 5: Page.goto: Proto...
5,CARRIER,4114413,ACTIVE,,10/01/2023,GRISEIDA MABEL BENAVIDES,1998 TRANSPORT,"4334 HWY 90 A\nALTAIR, TX 77412",(832) 724-6338,"5527 SWEETBRIAR ST\nHOUSTON, TX 77017-6423",https://safer.fmcsa.dot.gov/query.asp?searchty...,NaN
6,CARRIER,977543,ACTIVE,,09/12/2001,3 M TRANSPORT,,"3318 KY ROUTE 3379\nGRETHEL, KY 41631",(606) 587-2886,"PO BOX 62\nHAROLD, KY 41631",https://safer.fmcsa.dot.gov/query.asp?searchty...,NaN
7,CARRIER,929672,ACTIVE,"60,000 (2000)",,RONALD GRAHAM,3-G TRANSPORT,"568 E MCIVER RD\nFLORENCE, SC 29506-6925",(843) 678-4474,"568 E MCIVER RD\nFLORENCE, SC 29506-6925",https://safer.fmcsa.dot.gov/query.asp?searchty...,NaN
8,CARRIER,2936853,ACTIVE,1 (2020),01/13/2023,A & I TRANSPORT CORP,,"9890 NW 80TH PL\nHIALEAH, FL 33016",(786) 556-1478,"5355 W 8TH AVE\nHIALEAH, FL 33012",https://safer.fmcsa.dot.gov/query.asp?searchty...,NaN
9,CARRIER,1045774,ACTIVE,,08/01/2002,A A EAGLE PUMPING & DRAIN CLEANING INC,,"13121 ELEFF LANE\nHUDSON, FL 34669",(727) 869-8732,"POST OFFICE BOX 5486\nHUDSON, FL 34674",https://safer.fmcsa.dot.gov/query.asp?searchty...,NaN
